In [1]:
import re
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
#from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
tree = ET.parse('ABSA-15_Restaurants_Train_Final.xml')
root = tree.getroot()

In [2]:
skip = False
words = []
vecs = []
with open("fasttext150", "r") as f:
    for line in f:
        if skip:
            info = line.strip().split(' ')
            words.append(info[0]) 
            c = []
            for i in range(1, len(info)):
                c.append(float(info[i]))
            vecs.append(c)
        else:
            skip = True
npvecs = np.asarray(vecs, dtype='float32')

In [3]:
word_id = {}

vecs_lookup = np.concatenate([np.zeros((1, 100), dtype='float32'), npvecs], axis=0)
_id = 1
#counter = 0
for word in words:
    if word not in word_id:
        word_id[word] = _id
        #vecs_lookup.append(vecs[counter])
        #word_id[word] = [_id, vecs[counter]]
        _id += 1
#    counter += 1
#word_id["%NOWORD%"] = [0, np.zeros(100)] #NOWORD is for no word. id = 0. embedding = 0 0 0 0 0 0..
#vecs_lookup = np.insert(vecs_lookup, obj= 0, values = np.zeros(100), axis = 0) #id = 0 is reserved for embedding = 0 0 0 0 0 0 0 0...

#print(vecs_lookup[word_id['zero']])



In [4]:
sents = []
opins = []
for review in root:
    for sentence in review[0]:
        #sentence[0] #text
        if len(sentence) > 1:
            _ = re.sub(r"[\.\-\?\!,\"\']", " ", sentence[0].text.lower()).strip().split(' ')
            __ = []
            for word in _:
                if word != "":#get rid of ""s
                    #__.append(word)
                    if word in word_id:
                        __.append(word_id[word])
                    else:
                        __.append(0)
            if len(__) != 0:
                sents.append(__)
                opinions = [opin.attrib for opin in sentence[1]]
                opins.append(opinions)
            #data[sentence[0].text] = [opin.attrib for opin in sentence[1]]
        #print(sentence[1])


In [5]:
#print(shape(sents))
#print(sents[120:140], opins[120:140])
#print(sents[:14], opins[:14])

In [6]:
#print(zip(sents,opins))
_ = sorted(zip(sents,opins), key=lambda pair: len(pair[0]))
sents, opins = zip(*_)

#print(sents[120:140], opins[120:140]) #seems okay
#print(sents[:20], opins[:20])

In [7]:
#print(sents[:14], opins[:14])
#print(opins[14][0]['polarity'])
classes_labels = {'positive':1, 'negative':-1, 'neutral':0}
dumb_Y = []
for opin  in opins:
    _ = [target_op['polarity'] for target_op in opin]
    __ = np.asarray([classes_labels[class_] for class_ in _], dtype = 'int32')
    dumb_Y.append(round(np.mean(__, dtype = 'int32'))) #labels are crushed into a mean
        
        
print(opins[120:125], dumb_Y[120:125])
#print(word_id['amazing'])

([{'to': '10', 'target': 'waiter', 'from': '4', 'category': 'SERVICE#GENERAL', 'polarity': 'positive'}], [{'to': '18', 'target': 'place', 'from': '13', 'category': 'RESTAURANT#GENERAL', 'polarity': 'positive'}], [{'to': '0', 'target': 'NULL', 'from': '0', 'category': 'RESTAURANT#MISCELLANEOUS', 'polarity': 'positive'}], [{'to': '26', 'target': 'service', 'from': '19', 'category': 'SERVICE#GENERAL', 'polarity': 'positive'}], [{'to': '0', 'target': 'NULL', 'from': '0', 'category': 'SERVICE#GENERAL', 'polarity': 'positive'}]) [1, 1, 1, 1, 1]


In [8]:
def batch_maker(sents, dumb_Y, min_samples, sample_size, pos):
    #sample size not fixed!!!!!!!!!!!!!!
    batch = []
    y_batch = []
    
    while len(batch) < min_samples and pos < len(dumb_Y): #not really
        z = [0,0,0]
        #print("i got here pos = ", pos)
        #print(len(dumb_Y))
        index_to_set_1 = dumb_Y[pos]
        z[1+index_to_set_1] = 1 #one-hot dumb_Y prepared for ith sent
        
        if len(sents[pos]) <= sample_size:
            _ = sents[pos][0:]
            #while len(_) !=  sample_size:
            #    _.append(0)
            if len(_) != sample_size:
                _ = np.concatenate((_, np.zeros(sample_size - len(_), dtype = "int32")))
            y_batch.append(z)
            batch.append(np.asarray(_, dtype = "int32"))
            
            pos += 1 #sentence processed
        else:
            counter = 0
            while counter*sample_size < len(sents[pos]):
                _ = sents[pos][counter*sample_size:(counter+1)*sample_size]
                #while len(_) !=  sample_size:
                #    _.append(0)
                if len(_) != sample_size:
                    _ = np.concatenate((_, np.zeros(sample_size - len(_), dtype = "int32")))
                counter += 1
                y_batch.append(z)
                batch.append(np.asarray(_, dtype = "int32"))
                
            pos += 1 #sentence processed
    batch = np.asarray(batch)
    y_batch = np.asarray(y_batch)
    return batch, y_batch, pos

In [9]:
print(batch_maker(sents, dumb_Y, 4, 15, 0))
print(sents[0:7], dumb_Y[0:7])

i = 0
j = 0
k = 0
for a in dumb_Y:
    if a == 1:
        i += 1
    if a == -1:
        k += 1
    if a == 0:
        j += 1
print('+', i, '0', j, '-', k)

(array([[ 6348,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0],
       [ 7705,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0],
       [  642, 39524,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0]], dtype=int32), array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]]), 4)
([6348], [0], [7705], [642, 39524], [856, 4258], [3027, 1028], [0, 28883]) [1, 1, 1, 1, 1, 1, 1]
+ 762 0 114 - 244


In [10]:
train_amount = round(len(dumb_Y)*0.8)
train_sents = sents[:train_amount]
train_Y = dumb_Y[:train_amount]
test_sents = sents[train_amount:]
test_Y = dumb_Y[train_amount:]
print(len(train_Y), len(train_sents))

896 896


#### LSTM with return_sequences=True returns the hidden state of the LSTM for every timestep in the input to the LSTM. For example, if the input batch is (samples, timesteps, dims), then the call LSTM(units, return_sequences=True) will generate output of dimensions (samples, timesteps, units). https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer


In [11]:
# Training Parameters
learning_rate = 0.001
training_steps = 10
#batch_size = 128 NOSUCH
display_step = 1

# Network Parameters
num_input =  100
timesteps = 50 # timesteps ~ words?
num_hidden = 128 # hidden layer num of features
num_classes = 3
# tf Graph input
#X = tf.placeholder("float32", [None, timesteps, num_input])
X = tf.placeholder("int32", [None, timesteps])
Y = tf.placeholder("int32", [None, num_classes]) #one hot repr

In [12]:
word_embs = tf.constant(vecs_lookup, dtype=tf.float32) #words do have lookup too!

In [13]:
# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    'out': tf.Variable(tf.random_normal([2*num_hidden, num_classes])) #this is tf.float32
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [14]:
#tf.reset_default_graph()
def BiRNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)
    x = tf.nn.embedding_lookup(word_embs, x) #OKAY
    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32) #x, dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [15]:
logits = BiRNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


answers = tf.argmax(prediction, 1)
true_labels = tf.argmax(Y, 1)


# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [16]:
sess = tf.InteractiveSession()
sess.run(init)

#print(len(train_Y))
for step in range(1, training_steps+1):
    i = 0
    while i < len(train_sents):
        x_batch, y_batch, i = batch_maker(train_sents, train_Y, min_samples = 32, sample_size = timesteps, pos = i)
        #print(i)
        #x_lookup = tf.nn.embedding_lookup(word_embs, x_batch)
        #sess.run(x_lookup)
        sess.run(train_op, feed_dict={X: x_batch, Y: y_batch}) #x_lookup
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: x_batch, Y: y_batch}) #x_lookup
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")


Step 1, Minibatch Loss= 1.0277, Training Accuracy= 0.188
Step 1, Minibatch Loss= 0.9973, Training Accuracy= 0.125
Step 1, Minibatch Loss= 1.0228, Training Accuracy= 0.188
Step 1, Minibatch Loss= 0.8579, Training Accuracy= 0.094
Step 1, Minibatch Loss= 0.9059, Training Accuracy= 0.125
Step 1, Minibatch Loss= 0.8389, Training Accuracy= 0.656
Step 1, Minibatch Loss= 0.7802, Training Accuracy= 0.656
Step 1, Minibatch Loss= 0.7929, Training Accuracy= 0.812
Step 1, Minibatch Loss= 0.7767, Training Accuracy= 0.781
Step 1, Minibatch Loss= 0.7214, Training Accuracy= 0.750
Step 1, Minibatch Loss= 0.8263, Training Accuracy= 0.812
Step 1, Minibatch Loss= 0.9736, Training Accuracy= 0.531
Step 1, Minibatch Loss= 0.7590, Training Accuracy= 0.812
Step 1, Minibatch Loss= 0.7765, Training Accuracy= 0.750
Step 1, Minibatch Loss= 0.9541, Training Accuracy= 0.656
Step 1, Minibatch Loss= 0.9723, Training Accuracy= 0.688
Step 1, Minibatch Loss= 0.8401, Training Accuracy= 0.688
Step 1, Minibatch Loss= 0.8577,

Step 6, Minibatch Loss= 0.6209, Training Accuracy= 0.812
Step 6, Minibatch Loss= 0.7992, Training Accuracy= 0.656
Step 6, Minibatch Loss= 0.7656, Training Accuracy= 0.656
Step 6, Minibatch Loss= 0.5930, Training Accuracy= 0.812
Step 6, Minibatch Loss= 0.6328, Training Accuracy= 0.781
Step 6, Minibatch Loss= 0.6419, Training Accuracy= 0.750
Step 6, Minibatch Loss= 0.6510, Training Accuracy= 0.812
Step 6, Minibatch Loss= 1.0476, Training Accuracy= 0.531
Step 6, Minibatch Loss= 0.6222, Training Accuracy= 0.812
Step 6, Minibatch Loss= 0.7037, Training Accuracy= 0.750
Step 6, Minibatch Loss= 0.9166, Training Accuracy= 0.656
Step 6, Minibatch Loss= 0.9017, Training Accuracy= 0.688
Step 6, Minibatch Loss= 0.8148, Training Accuracy= 0.688
Step 6, Minibatch Loss= 0.8024, Training Accuracy= 0.719
Step 6, Minibatch Loss= 0.7343, Training Accuracy= 0.750
Step 6, Minibatch Loss= 1.1460, Training Accuracy= 0.531
Step 6, Minibatch Loss= 0.7194, Training Accuracy= 0.719
Step 6, Minibatch Loss= 0.9076,

In [17]:
preds = np.array([])
i = 0
while i < len(test_sents):
    x_batch, y_batch, i = batch_maker(test_sents, test_Y, min_samples = 32, sample_size = timesteps, pos = i)
        #print(i, x_batch.shape, y_batch.shape)
        #x_lookup = tf.nn.embedding_lookup(word_embs, x_batch)
        #sess.run(x_lookup)
        #sess.run(train_op, feed_dict={X: x_batch, Y: y_batch}) #x_lookup
        #print('HUY')
             
    pred = sess.run(correct_pred, feed_dict={X: x_batch, Y: y_batch}) #x_lookup
    preds = np.append(preds, np.asarray(pred))
print(np.mean(preds))

0.5288888888888889


In [19]:
all_answers = np.array([])
all_true_labels = np.array([])
i = 0

while i < len(test_sents):
    x_batch, y_batch, i = batch_maker(test_sents, test_Y, min_samples = 32, sample_size = timesteps, pos = i)
    
    a, t = sess.run([answers, true_labels], feed_dict={X: x_batch, Y: y_batch})
    #print(a,t)
    all_answers = np.append(all_answers, np.asarray(a)) #no need to flatten
    all_true_labels = np.append(all_true_labels, np.asarray(t))
    
    
TP, FP, FN = {0:0, 1:0, 2:0}, {0:0, 1:0, 2:0}, {0:0, 1:0, 2:0}
for l in range(3):
    for counter in range(len(all_answers)):
        if all_answers[counter] == all_true_labels[counter] and all_answers[counter] == l: #answer matches label
            TP[l] += 1
        if all_answers[counter] != all_true_labels[counter] and all_answers[counter] == l:
            FP[l] += 1
        if all_answers[counter] != all_true_labels[counter] and all_answers[counter] != l:
            FN[l] += 1
            
Prec, Rec = {}, {}
for l in range(3):
    try:
        Prec[l] = TP[l]/float((TP[l]+FP[l]))
    except Exception:
        Prec[l] = 0
    try:
        Rec[l] = TP[l]/float((TP[l]+FN[l]))
    except Exception:
        Rec[l] = 0
F1 = {}
for l in range(3):
    try:
        F1[l] = 2*Prec[l]*Rec[l]/(Prec[l]+Rec[l])
    except Exception:
        F1[l] = 0
    
    
MF1 = sum(_ for _ in F1.values())/3
print(F1, MF1)
print(TP, FP, FN, Prec, Rec)


{0: 0, 1: 0, 2: 0.691860465116279} 0.23062015503875966
{0: 0, 1: 0, 2: 119} {0: 0, 1: 0, 2: 106} {0: 106, 1: 106, 2: 0} {0: 0, 1: 0, 2: 0.5288888888888889} {0: 0.0, 1: 0.0, 2: 1.0}


In [ ]:
sess.close()

In [ ]:
tf.reset_default_graph()